In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Importing necessary libraries for data analysis and visualization

import json
from datetime import datetime

import matplotlib.pyplot as plt  # For creating various plots and visualizations
import numpy as np
import pandas as pd  # For handling and manipulating structured data (tables, CSV, etc.)
import seaborn as sns  # For advanced data visualization with statistical capabilities
# Importing automatic data profiling tool
import ydata_profiling as yd  # Used to generate a report with statistics, correlations, and distributions for data exploration
# Importing time series analysis tools from pandas and statsmodels
from pandas.plotting import lag_plot  # For visualizing lag correlations in time series data
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Importing seasonal decomposition tool for time series analysis
from statsmodels.tsa.seasonal import  seasonal_decompose  # To decompose a time series into trend, seasonality, and residual components
# Importing statistical test for stationarity
from statsmodels.tsa.stattools import  adfuller  # Augmented Dickey-Fuller (ADF) test to check stationarity of a time series

In [2]:
%matplotlib inline
from pandas.api.types import is_datetime64_any_dtype



In [ ]:
# Load the dataset


In [5]:
df = pd.read_csv(
    "../datasets/energydata_complete.csv",
    parse_dates=True,
)
# Try to detect a datetime column
for col in df.columns:
    df[col] = pd.to_datetime(
        df[col], errors="coerce"
    )  # Convert to datetime if possible
    if is_datetime64_any_dtype(df[col]):
        df.set_index(col, inplace=True)
        print(f"Set '{col}' as the datetime index.")
        break 
df

Set 'date' as the datetime index.


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812
2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940
2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117


In [6]:
DataProfile = yd.ProfileReport(df)
# DataProfile.to_file("Profile.json")
DataProfile.to_file("Profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Basic exploration
print(df.head())
print(df.info())
print(df.describe())



In [ ]:
def handle_missing_values(df, validation_fraction=0.1, random_state=42):
    """
    Applies the best missing value imputation strategy based on MSE.
    
    This function creates a validation set by artificially masking a fraction of 
    non-missing values. It then compares several imputation methods by computing 
    the mean squared error (MSE) between the imputed values and the true values 
    at the masked positions. The method with the lowest MSE is applied to the 
    original DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame with possible missing values.
    validation_fraction (float): Fraction of non-missing values to mask for validation.
    random_state (int): Seed for reproducibility.
    
    Returns:
    pd.DataFrame: The DataFrame with missing values imputed.
    """
    # If no missing values, simply return the original dataframe.
    if df.isnull().sum().sum() == 0:
        print("No missing values detected after EDA. Skipping handling.")
        return df
    print("Missing values detected. Selecting best imputation method.")

    # Create a copy of df to simulate additional missingness for validation.
    df_validation = df.copy()
    # This mask DataFrame will track the positions that are artificially set to NaN.
    mask = pd.DataFrame(False, index=df.index, columns=df.columns)
    # For each column, randomly mask a fraction of originally non-missing values.
    np.random.seed(random_state)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df_validation[col] = pd.to_numeric(df_validation[col], errors='coerce')
        non_missing_indices = df[df[col].notnull()].index
        n_to_mask = int(len(non_missing_indices) * validation_fraction)
        if n_to_mask > 0:
            masked_indices = np.random.choice(non_missing_indices, n_to_mask, replace=False)
            mask.loc[masked_indices, col] = True
            df_validation.loc[masked_indices, col] = np.nan

    # Define candidate imputation methods applied on the artificially masked dataframe.
    imputed_dfs = {
        "forward_fill": df_validation.fillna(method='ffill'),
        "backward_fill": df_validation.fillna(method='bfill'),
        "mean_imputation": df_validation.fillna(df_validation.mean()),
        "median_imputation": df_validation.fillna(df_validation.median()),
        "interpolation": df_validation.interpolate()
    }

    mse_scores = {}
    # Compute MSE for each method on the positions we masked.
    for method_name, imputed_df in imputed_dfs.items():
        # y_true holds the ground truth values from the original df where we masked values.
        y_true = df[mask]
        # y_pred holds the corresponding imputed values.
        y_pred = imputed_df[mask]
        mse = np.mean((y_true - y_pred) ** 2)
        mse_scores[method_name] = mse
    print(mse_scores)
    best_method = min(mse_scores, key=mse_scores.get)
    print(f"Selected best missing value handling method: {best_method}")

    # Now apply the best method on the original dataframe with actual missing values.
    if best_method == "forward_fill":
        return df.fillna(method='ffill')
    elif best_method == "backward_fill":
        return df.fillna(method='bfill')
    elif best_method == "mean_imputation":
        return df.fillna(df.mean())
    elif best_method == "median_imputation":
        return df.fillna(df.median())
    elif best_method == "interpolation":
        return df.interpolate()
    else:
        print("No valid imputation method selected, returning original df.")
        return df

'''
def convert_if_majority_numeric(dataframe):
    # print(dataframe)
    df1 = dataframe
    df1.dropna(inplace=True)
    """Convert string numbers to numeric values only if most elements are numeric."""
    for column in df1.columns:
        type_counts = df[column].apply(lambda x: is_numeric_dtype(x)).value_counts()
        for x in df[column]:
            print(type(x))

        # Check if majority is numeric
        majority_is_numeric = type_counts.get(True, 0) > type_counts.get(False, 0)
        print(column, " : ", type_counts.get(True, 0))
        print(
            column,
            " : ",
        )
        print(column, " : ", majority_is_numeric)

        if majority_is_numeric:
            df[column] = pd.to_numeric(
                df[column],
                errors="coerce",
            )  # Convert only if majority is numeric
            print("converted")
'''

In [ ]:
df = handle_missing_values(df)
def generate_mlops_report(df):
    """
    Generate a machine-interpretable EDA report for MLOps preprocessing
    
    Parameters:
    df (pandas.DataFrame): Input time series dataframe
    
    Returns:
    dict: Structured, machine-readable analysis report
    """
    report = {
        "dataset_metadata": {
            "total_columns": len(df.columns),
            "total_rows": len(df),
            "date_range": {
                "start": str(df.index.min()),
                "end": str(df.index.max())
            }
        },
        "preprocessing_recommendations": {
            "stationarity": {},
            "feature_scaling": [],
            "feature_engineering": []
        },
        "statistical_insights": {
            "descriptive_stats": {},
            "correlations": {
                "significant_correlations": [],
                "correlation_matrix": {}
            }
        }
    }
    
    # Stationarity Analysis
    for column in df.columns:
        adf_result = adfuller(df[column])
        report["preprocessing_recommendations"]["stationarity"][column] = {
            "is_stationary": str(adf_result[1] < 0.05),
            "p_value": float(adf_result[1]),
            "transformation_needed": "YES" if adf_result[1] >= 0.05 else "NO"
        }
        
        # Recommend transformations if not stationary
        if adf_result[1] >= 0.05:
            report["preprocessing_recommendations"]["feature_engineering"].append({
                "column": column,
                "suggested_transformations": [
                    "log_transformation",
                    "differencing",
                    "rolling_mean_normalization"
                ]
            })
    
    # Descriptive Statistics
    for column in df.columns:
        report["statistical_insights"]["descriptive_stats"][column] = {
            "mean": float(df[column].mean()),
            "std": float(df[column].std()),
            "min": float(df[column].min()),
            "max": float(df[column].max())
        }
    
    # Correlation Analysis
    corr_matrix = df.corr()
    report["statistical_insights"]["correlations"]["correlation_matrix"] = \
        {str(col): {str(subcol): float(corr_matrix.loc[col, subcol]) 
                    for subcol in df.columns} 
         for col in df.columns}
    
    # Significant Correlations
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2:
                correlation = float(corr_matrix.loc[col1, col2])
                if abs(correlation) > 0.5:
                    report["statistical_insights"]["correlations"]["significant_correlations"].append({
                        "features": [col1, col2],
                        "correlation": correlation,
                        "strength": "strong" if abs(correlation) > 0.7 else "moderate"
                    })
    
    # Feature Scaling Recommendations
    for column in df.columns:
        if df[column].std() > 1:  # Suggest scaling for features with high variance
            report["preprocessing_recommendations"]["feature_scaling"].append({
                "column": column,
                "recommended_method": ["standardization", "min_max_scaling"]
            })
    
    return report

# Generate and save the report
final_report = generate_mlops_report(df)

# Save to JSON
with open('mlops_eda_report.json', 'w') as f:
    json.dump(final_report, f, indent=4)

print("MLOps-friendly EDA Report generated and saved to 'mlops_eda_report.json'")

In [ ]:
for column in df.columns:
    plt.figure(df.columns.get_loc(column)+1,figsize=(18, 18))
    # Plot the time series data for the current column
    plt.plot(df.index, df[column], label=column)
plt.title('Time Series Plot')
plt.xlabel('Date')
plt.ylabel('Values')
plt.legend()
plt.show()


In [ ]:

# Loop through each column in the DataFrame to perform stationarity testing and seasonal decomposition
for column in df.columns:
    print(f"\nAnalyzing column: {column}")  # Print the name of the column being analyzed
    
    # Create a new figure for the decomposition plot
    plt.figure(figsize=(20, 14))

    # --- Augmented Dickey-Fuller (ADF) Test for Stationarity ---
    adf_result = adfuller(df[column])  # Perform the ADF test on the current column
    print(f"ADF Statistic for {column}: {adf_result[0]}")  # Print the test statistic
    print(f"p-value for {column}: {adf_result[1]}")  # Print the p-value (to check stationarity)
    
    # The ADF test helps determine if a time series is stationary.
    # If p-value < 0.05, we reject the null hypothesis and conclude that the series is stationary.

    # --- Seasonal Decomposition ---
    # Decompose the time series into trend, seasonality, and residuals using an additive model
    decomposition = seasonal_decompose(df[column], model='additive', period=12)
    
    # Plot the decomposition results (observed, trend, seasonal, and residual components)
    decomposition.plot()
    
    # Set a title for the decomposition plot
    plt.suptitle(f'Seasonal Decomposition of {column}')
    
    # Display the plots
    plt.show()


In [ ]:
# Compute the rolling mean with a 12-month window for smoothing time series data
rolling_means = df.rolling(window=12).mean()

# Create a new figure with a specific size
plt.figure(figsize=(12, 6))

# Loop through each column in the DataFrame to plot its rolling mean
for column in df.columns:
    plt.plot(rolling_means.index, rolling_means[column], label=f'{column} Rolling Mean')

# Set the title of the plot
plt.title('Rolling Mean (12-month window)')

# Add a legend to indicate which rolling mean corresponds to which column
plt.legend()

# Display the plot
plt.show()



In [ ]:
# Compute the correlation matrix for all numerical columns in the DataFrame
df_corr = df.corr()

# Print the correlation matrix to inspect the numerical relationships between variables
print("Correlation Matrix:")
print(df_corr)

# Create a heatmap to visualize the correlation matrix
sns.heatmap(df_corr, annot=True, cmap='coolwarm')

# Set the title for the heatmap
plt.title('Correlation Heatmap')

# Display the heatmap
plt.show()



In [ ]:
for column in df.columns:
    plt.figure()
    lag_plot(df[column])
    plt.title(f'Lag Plot for {column}')
    plt.show()


In [ ]:
for column in df.columns:
    print(f"Autocorrelation and Partial Autocorrelation for {column}")
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    plot_acf(df[column], ax=axes[0], lags=20, title=f'ACF: {column}')
    plot_pacf(df[column], ax=axes[1], lags=20, title=f'PACF: {column}')
    plt.show()

In [ ]:
# Create a new DataFrame with lagged versions of the original columns
lagged_df = pd.concat(
    [df.shift(i).add_suffix(f"_lag{i}") for i in range(1, 4)], axis=1  # Lag 1, 2, 3
)

# Concatenate the original DataFrame with the lagged features and remove any NaN values
lagged_df = pd.concat([df, lagged_df], axis=1).dropna()

# Display the first few rows of the new DataFrame with lagged values
print(lagged_df.head())


In [ ]:
# Create pairwise scatter plots for all numerical columns in the DataFrame
sns.pairplot(df, kind="reg")

plt.show()